In [3]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline
import pymongo
import io
import pandas as pd
import pickle
import datetime
import time
import gzip
import lzma
import pytz
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np

dd = '20201223'

pd.set_option('max_rows', 200)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startTm = datetime.datetime.now()

startDate = dd
endDate = dd

readPath = '\\\\192.168.10.28\\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

thisDate = dateLs[0]
assert(thisDate == dd)

readPath = '\\\\192.168.10.28\\equityTradeLogs'
rawOrderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%thisDate))
for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0)
    rawOrderLog[col] = rawOrderLog[col].astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)
original_data = rawOrderLog.copy()

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

display('There are accounts with duplicated ticks:')
display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    print('There are orders in 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
rawOrderLog = pd.concat([r1, r2])
del r1
del r2  


rawOrderLog = rawOrderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
rawOrderLog['clock'] = rawOrderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
rawOrderLog["broker"] = np.where(rawOrderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), rawOrderLog['accCode'] // 10000, rawOrderLog['accCode'] // 100)
rawOrderLog['colo_broker'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['broker'].astype('str')
rawOrderLog['colo_account'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['accCode'].astype('str')
rawOrderLog['order'] = rawOrderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
rawOrderLog['group'] = rawOrderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
rawOrderLog['startClock'] = rawOrderLog.groupby(['order'])['caamd'].transform('first')
rawOrderLog['duration'] = rawOrderLog['clockAtArrival'] - rawOrderLog['startClock']
rawOrderLog['orderPrice'] = rawOrderLog['orderPrice'].apply(lambda x: round(x, 2))
rawOrderLog['tradePrice'] = rawOrderLog['tradePrice'].apply(lambda x: round(x, 2))
rawOrderLog['orderDirection1'] = np.where(rawOrderLog["orderDirection"] == -2, -1, np.where(
    rawOrderLog["orderDirection"] == 2, 1, rawOrderLog["orderDirection"]))
rawOrderLog['sdd'] = rawOrderLog.groupby('order')['sdd'].transform('first')
rawOrderLog['caamd'] = rawOrderLog.groupby('order')['caamd'].transform('first')
rawOrderLog["ars"] = rawOrderLog.groupby(['order'])['ars'].transform('first')
orderLog = rawOrderLog.copy()

print('# of orders in raw orderLog')
display(rawOrderLog['order'].nunique())

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()
orderLog['firstUpdateType'] = orderLog.groupby(['order'])['updateType'].transform('first')

print('1. delete orders firstUpdateType != 0')
print(orderLog[orderLog['firstUpdateType'] != 0]['order'].nunique())

orderLog = orderLog[orderLog['firstUpdateType'] == 0]
orderLog['insertNum'] = np.where(orderLog['updateType'] == 0, 1, 0)
orderLog['insertNum'] = orderLog.groupby(['order'])['insertNum'].transform('sum')

print('2. delete orders more than 2 updateType == 0')
print(orderLog[orderLog['insertNum'] != 1]['order'].nunique())

orderLog = orderLog[orderLog['insertNum'] == 1]
orderLog = orderLog[orderLog['secid'] >= 2000000].reset_index(drop=True)
def getTuple(x):
    return tuple(i for i in x)

# 1. market orders
savePath = 'L:\\orderLog\\mdData'
rawMsgDataSZ = pd.read_pickle(os.path.join(savePath, 'mdLog_msg_%s.pkl'%dd))
orderDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '2'][['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'OrderQty', 'Price', 'cum_volume', "TransactTime"]].reset_index(drop=True)
orderDataSZ['updateType'] = 0
tradeDataSZ = pd.concat([rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'BidApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]],
                         rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]], sort=False)
tradeDataSZ['ApplSeqNum'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), tradeDataSZ['OfferApplSeqNum'], tradeDataSZ['BidApplSeqNum'])
tradeDataSZ['Side'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), 2, 1)
tradeDataSZ = tradeDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]
tradeDataSZ['updateType'] = 4
cancelDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '4'][['SecurityID', 'BidApplSeqNum', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]].reset_index(drop=True)
cancelDataSZ['ApplSeqNum'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, cancelDataSZ['OfferApplSeqNum'], cancelDataSZ['BidApplSeqNum'])
cancelDataSZ['Side'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, 2, 1)
cancelDataSZ = cancelDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradeQty', 'cum_volume', "TransactTime"]]
cancelDataSZ['updateType'] = 3

msgDataSZ = pd.concat([orderDataSZ, tradeDataSZ, cancelDataSZ], sort=False)
del orderDataSZ
del tradeDataSZ
del cancelDataSZ
msgDataSZ = msgDataSZ.sort_values(by=['SecurityID', 'ApplSeqNum', 'sequenceNo']).reset_index(drop=True)
msgDataSZ['TradePrice'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradePrice'], 0)
msgDataSZ['TradePrice'] = msgDataSZ['TradePrice'].astype('int64')
msgDataSZ['TradeQty'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradeQty'], 0)
msgDataSZ['TradeQty'] = msgDataSZ['TradeQty'].astype('int64')
msgDataSZ['secid'] = msgDataSZ['SecurityID'] + 2000000
assert(msgDataSZ['ApplSeqNum'].max() < 1e8)
msgDataSZ['StockSeqNum'] = msgDataSZ['SecurityID']*1e8 + msgDataSZ['ApplSeqNum']
msgDataSZ['date'] = int(thisDate) 
print('finish market orders')


# 2. orderLog
infoData = orderLog[(orderLog['date'] == int(thisDate)) & (orderLog["isMsg"] == 1) 
                    & (orderLog['updateType'].isin([0, 3, 4]))].reset_index(drop=True)
infoData['Price'] = infoData['orderPrice'].apply(lambda x: round(x*100, 0))
infoData['Price'] = infoData['Price'].astype('int64')*100
infoData['OrderQty'] = infoData['absOrderSize']
infoData['Side'] = np.where(infoData['orderDirection1'] == 1, 1, 2)
infoData['TradePrice'] = np.where(infoData['updateType'] == 4, round(infoData['tradePrice']*100, 0), 0)
infoData['TradePrice'] = infoData['TradePrice'].astype('int64')*100
statusInfo = infoData.groupby(['order'])['updateType'].apply(lambda x: tuple(x)).reset_index()
statusInfo.columns = ['order', 'statusLs']
tradePriceInfo = infoData.groupby(['order'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
tradePriceInfo.columns = ['order', 'TradePriceLs']
tradeQtyInfo = infoData.groupby(['order'])['absFilledThisUpdate'].apply(lambda x: tuple(x)).reset_index()
tradeQtyInfo.columns = ['order', 'TradeQtyLs']
infoData = infoData[infoData['updateType'] == 0]
infoData = pd.merge(infoData, statusInfo, how='left', on=['order'], validate='one_to_one')
infoData = pd.merge(infoData, tradePriceInfo, how='left', on=['order'], validate='one_to_one')
infoData = pd.merge(infoData, tradeQtyInfo, how='left', on=['order'], validate='one_to_one')
infoData['brokerNum'] = infoData.groupby(['date', 'secid', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'])['colo_account'].transform('count')
display(infoData[infoData['brokerNum'] >= 2].groupby(['colo', 'accCode'])['date'].count())
display('%.2f%%'%(infoData[infoData['brokerNum'] >= 2].shape[0] / infoData.shape[0]*100))
display(infoData[infoData['brokerNum'] >= 2].shape[0])
display(infoData.shape[0])
infoData = infoData[infoData['brokerNum'] == 1]
infoData = infoData[['date', 'secid', 'vai', 'ars', "mrstaat", "mrstauc", 'exchange', 'group', 'Price', 'OrderQty', 
                     'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum', 'order', 'colo', 'accCode', 
                     'clockAtArrival', 'orderDirection', 'absOrderSize']]   

print('finish our orders')

# 3. find the position of our orders
checkLog = msgDataSZ[msgDataSZ['updateType'] == 0]

checkLog = pd.merge(checkLog, infoData.drop_duplicates(subset=['secid', 'ApplSeqNum'])[['secid', 'ApplSeqNum', 'ars']], 
                on=['secid', 'ApplSeqNum'], how='left')
checkLog['ApplSeqNum'] = np.where(~checkLog['ars'].isnull(), checkLog['ApplSeqNum'], checkLog['ars'])
checkLog['start_time'] = np.where(~checkLog['ars'].isnull(), checkLog['clockAtArrival'], checkLog['ars'])
checkLog.drop(["ars"],axis=1,inplace=True)
checkLog['ApplSeqNum'] = checkLog.groupby(['secid'])['ApplSeqNum'].ffill()
checkLog['start_time'] = checkLog.groupby(['secid'])['start_time'].ffill()
checkLog['end_time'] = checkLog['start_time'] + 100*1e3
checkLog = checkLog[(~checkLog['ApplSeqNum'].isnull()) & (checkLog['clockAtArrival'] > checkLog['start_time']) & 
                   (checkLog['clockAtArrival'] < checkLog['end_time'])]
msgDataSZ = msgDataSZ[msgDataSZ['StockSeqNum'].isin(checkLog['StockSeqNum'].values)]
print('finish get the interval')

statusInfo = msgDataSZ.groupby(['StockSeqNum'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
statusInfo.columns = ['StockSeqNum', 'statusLs']
tradePriceInfo = msgDataSZ.groupby(['StockSeqNum'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
tradePriceInfo.columns = ['StockSeqNum', 'TradePriceLs']
tradeQtyInfo = msgDataSZ.groupby(['StockSeqNum'])['TradeQty'].apply(lambda x: tuple(x)).reset_index()
tradeQtyInfo.columns = ['StockSeqNum', 'TradeQtyLs']
del msgDataSZ

checkLog = pd.merge(checkLog, statusInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
checkLog = pd.merge(checkLog, tradePriceInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
checkLog = pd.merge(checkLog, tradeQtyInfo, how='left', on=['StockSeqNum'], validate='one_to_one')  
del statusInfo
del tradePriceInfo
del tradeQtyInfo
checkLog = checkLog.rename(columns={'clockAtArrival':'caa_orderLog'})

try:
    checkLog = pd.merge(checkLog, infoData, how='left', on=['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], validate='many_to_one')
except:
    print('There are orders with same pattern but different vai, same ApplSeqNum')
    display([infoData[infoData.duplicated(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)]])
    checkLog = pd.merge(checkLog, infoData, how='left', on=['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'])
del infoData

checkLog = checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkLog = checkLog[~checkLog['accCode'].isnull()]
checkLog = checkLog.reset_index(drop=True)

finish market orders


In [4]:
test = orderLog[orderLog['updateType'].isin([0, 4, 3])]
display(checkLog.shape[0])
test = pd.merge(checkLog[['date', 'secid', 'order', 'start_time', 'accCode', 'StockSeqNum']], test, on=['date', 'secid', 'order', 'accCode'], how='inner')
display(test[test['updateType'] == 0].shape[0])
test = test.sort_values(by=['order', 'clockAtArrival'])
msgDataSZ = msgDataSZ.sort_values(by=['StockSeqNum', 'clockAtArrival'])
test['innerSeq'] = test.groupby('order')['date'].cumcount()
msgDataSZ['innerSeq'] = msgDataSZ.groupby('StockSeqNum')['date'].cumcount()
test = pd.merge(msgDataSZ[['StockSeqNum', 'clockAtArrival', 'updateType', 'innerSeq']], test, on=['StockSeqNum', 'updateType', 'innerSeq'], how='inner')
display(test[test['updateType'] == 0].shape[0])

85257

85257

85257

In [5]:
test1 = test[test['updateType'] == 4]

In [6]:
test1['caa_orderLog'] = test1['clockAtArrival_y'] - test1['startClock'] + test1['start_time']
test1['caa_market'] = test1['clockAtArrival_x']
(test1['caa_market'] - test1['caa_orderLog']).describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]).astype('int64')

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


count     136267
mean       -1708
std        18710
min     -2173663
10%        -2787
20%         -696
30%         -469
40%         -349
50%         -258
60%         -175
70%          -97
80%          -12
90%          103
95%          235
99%          733
max      1105637
dtype: int64

In [15]:
test2 = test1[test1['caa_market'] - test1['caa_orderLog'] < 0]
(test2['caa_market'] - test2['caa_orderLog'] ).describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]).astype('int64')

count     110705
mean       -2324
std        17465
min     -2173663
10%        -4125
20%         -938
30%         -568
40%         -434
50%         -343
60%         -269
70%         -201
80%         -136
90%          -72
95%          -38
99%           -8
max           -1
dtype: int64

In [13]:
test1[test1['caa_market'] - test1['caa_orderLog'] == -2173663]['start_time'].iloc[0]

1608694003957365.0

In [12]:
test1[test1['caa_market'] - test1['caa_orderLog'] == -2173663][['clockAtArrival_x', 'clockAtArrival_y', 'startClock', 'start_time', 'caa_market', 'caa_orderLog']]

,clockAtArrival_x,clockAtArrival_y,startClock,start_time,caa_market,caa_orderLog
107553,1608694003957370,1608694003637829,1608694001464161,1.608694e+15,1608694003957370,1.608694e+15


In [8]:
orderLog[orderLog['order'] == 986][['date', 'colo', 'accCode', 'ApplSeqNum', 'updateType', 'secid', 'clockAtArrival', 'caamd', 'startClock', 'orderPrice', 'absOrderSize', 'tradePrice', 'absFilledThisUpdate']]

,date,colo,accCode,ApplSeqNum,updateType,secid,clockAtArrival,caamd,startClock,orderPrice,absOrderSize,tradePrice,absFilledThisUpdate
3922,20201223,zs_52_06,5222,14057670.0,0,2002778,1608694001464213,1608694001464161,1608694001464161,26.34,300,-1.00,0
3923,20201223,zs_52_06,5222,0.0,2,2002778,1608694001466960,1608694001464161,1608694001464161,26.34,300,-1.00,0
3924,20201223,zs_52_06,5222,0.0,4,2002778,1608694003637829,1608694001464161,1608694001464161,26.34,300,26.34,300


In [11]:
msgDataSZ[(msgDataSZ['ApplSeqNum'] == 14057670) & (msgDataSZ['SecurityID'] == 2778)]

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,innerSeq
62222479,2778,14057670.0,1608694003957365,85975080,1.0,300.0,263400.0,625000.0,112643150,0,0,0,2002778,2.778141e+11,20201223,0
62222480,2778,14057670.0,1608694003957367,85975081,1.0,NaN,NaN,625300.0,112643150,4,263400,300,2002778,2.778141e+11,20201223,1


In [14]:
1608694003957370 - (1608694003637829 - 1608694001464161 + 1608694003957365)

-2173663

In [10]:
msgDataSZ[msgDataSZ['StockSeqNum'] == 277814057672]

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,innerSeq
62222481,2778,14057672.0,1608694003957368,85975082,1.0,300.0,263400.0,625300.0,112643150,0,0,0,2002778,2.778141e+11,20201223,0
62222482,2778,14057672.0,1608694003957370,85975083,1.0,NaN,NaN,625600.0,112643150,4,263400,300,2002778,2.778141e+11,20201223,1
